<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-предварительный-анализ-данных" data-toc-modified-id="Загрузка-и-предварительный-анализ-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и предварительный анализ данных</a></span></li><li><span><a href="#Токенизация-и-лемматизация-комментариев" data-toc-modified-id="Токенизация-и-лемматизация-комментариев-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Токенизация и лемматизация комментариев</a></span></li><li><span><a href="#Разделение-исходных-данных-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разделение-исходных-данных-на-обучающую-и-тестовую-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разделение исходных данных на обучающую и тестовую выборки</a></span></li><li><span><a href="#Векторизация-текстовых-признаков" data-toc-modified-id="Векторизация-текстовых-признаков-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Векторизация текстовых признаков</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-выбор-наилучшей-модели" data-toc-modified-id="Обучение-и-выбор-наилучшей-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и выбор наилучшей модели</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регресии" data-toc-modified-id="Модель-логистической-регресии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регресии</a></span></li><li><span><a href="#Модель-линейной-классификации-методом-опорных-векторов" data-toc-modified-id="Модель-линейной-классификации-методом-опорных-векторов-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель линейной классификации методом опорных векторов</a></span></li><li><span><a href="#Модель-решающего-дерева" data-toc-modified-id="Модель-решающего-дерева-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель решающего дерева</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-градиентного-бустинга-на-базе-LightGBM" data-toc-modified-id="Модель-градиентного-бустинга-на-базе-LightGBM-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Модель градиентного бустинга на базе LightGBM</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-наилучшей-модели" data-toc-modified-id="Тестирование-наилучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование наилучшей модели</a></span><ul class="toc-item"><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Система классификации текстовых комментариев для интерент-магазина

Интернет-магазин запускает новый сервис, c помощью которого пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах, то есть могут предлагать свои правки и комментировать изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо создать систему для классифицировать комментариев на позитивные и негативные. В распоряжении имеется набор данных с разметкой о токсичности правок.

Необходимая метрика качества модели: *F1* - не меньше 0.75. 

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка данных

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import warnings

from lightgbm import LGBMClassifier
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
warnings.filterwarnings("ignore")

### Загрузка и предварительный анализ данных

Произведем загрузку данных.

In [3]:
df = pd.read_csv(r'C:\Users\lebeda\Yandex_Practicum\Yandex_projects\data_sets\prjkt_13(sp16)\toxic_comments.csv') 

Проведем предварительный осмотр данных на предмет наличия артефактов и аномалий.

In [4]:
df.shape #размерность

(159571, 2)

In [5]:
df.info() #вывод сводной информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df.head() 

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.isna().sum() #проверка на наличие пропусков

text     0
toxic    0
dtype: int64

In [8]:
df.duplicated().sum() #проверка на наличие полных дубликатов строк

0

In [9]:
df.describe() #получение описательных характеристик

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [10]:
df.query('toxic == 1')['toxic'].count() #колличество комметариев с указанным признаком "токсичности"

16225

In [11]:
df.query('toxic == 0')['toxic'].count() #колличество комметариев, в которых признак "токсичности" не указан

143346

**Вывод:** Предоставленный датасет - таблица из 159570 строк и двух столбцов, содержащая тексты комметариев на английском языке в одном столбце ("text") и размеченное (заранее определенное) значение признака "токсичности" в другом столбце ("toxic"). Пропуски и полные дубликаты отсутсвуют. Доля "токсичных" комментариев от общего числа сотавляет около 11%, что необходимо принять во внимание для разделение исходных данных на выборки для последующего построения моделей обучения.

### Токенизация и лемматизация комментариев

Для получения вектора признаков на основе весов важности слов проведем предообработку текста, разделив его на токены в виде слов и найдя их леммы.

Для работы над таблицей будем использовать копию экземпляра исходных данных.

In [12]:
df_work = df.copy()

Для предобработки текста создадим ряд функций, позволяющих модифицировать исходный массив.

In [13]:
def re_out(text): #функция выделения из текста регулярных выражений (только латиница в верхнем и нижнем регистрах)
    return " ".join((re.sub(r'[^a-zA-z]', ' ', text)).split())

In [14]:
def tokinizer(text): #функция токенизации текста (токенизация по словам)
    return word_tokenize(text)

In [15]:
def get_wordnet_pos(word): #функция опредления POS-тега лемматизатора для более точной лемматзации
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [16]:
lemmatizer = WordNetLemmatizer() #создание класса лемматизации 

def lemmatization(text): #функция лематизации с учетом опеделенного POS-тега
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text])

Выделим из комметариев регулярные выражения.

In [17]:
df_work['lemmas'] = df_work['text'].apply(re_out)

In [18]:
df_work.tail()

,text,toxic,lemmas
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,And it looks like it was actually you who put ...
159570,"""\nAnd ... I really don't think you understand...",0,And I really don t think you understand I came...


Проведем токенезацию комментариев по словам.

In [19]:
tqdm.pandas(desc="progress bar!")

In [20]:
df_work['lemmas'] = df_work['lemmas'].progress_apply(tokinizer)

progress bar!: 100%|██████████| 159571/159571 [00:30<00:00, 5318.96it/s]


In [21]:
df_work.tail()

,text,toxic,lemmas
159566,""":::::And for the second time of asking, when ...",0,"[And, for, the, second, time, of, asking, when..."
159567,You should be ashamed of yourself \n\nThat is ...,0,"[You, should, be, ashamed, of, yourself, That,..."
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,"[Spitzer, Umm, theres, no, actual, article, fo..."
159569,And it looks like it was actually you who put ...,0,"[And, it, looks, like, it, was, actually, you,..."
159570,"""\nAnd ... I really don't think you understand...",0,"[And, I, really, don, t, think, you, understan..."


Проведем лемматизацию комментариев с учетом определяемого POS-тега токена.

In [22]:
df_work['lemmas'] = df_work['lemmas'].progress_apply(lemmatization)

progress bar!: 100%|██████████| 159571/159571 [53:16<00:00, 49.93it/s] 


In [23]:
df_work.tail()

,text,toxic,lemmas
159566,""":::::And for the second time of asking, when ...",0,And for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...
159570,"""\nAnd ... I really don't think you understand...",0,And I really don t think you understand I come...


### Разделение исходных данных на обучающую и тестовую выборки

Проведем разделение полученной таблицы с леммами и признаками "токсичности" на тренировочную и тестовую части в соотношении 4/1 соответственно. Разделение выполним с признаком стратификации данных по целевому столбцу. 

In [24]:
#разделение таблицы на тренировочную и тестовую части
lemma_train, lemma_test, target_train, target_test = train_test_split(df_work['lemmas'], df_work['toxic'],
                                                                      stratify=df_work['toxic'],
                                                                      test_size=0.2)

In [25]:
lemma_train.shape #проверка размерности тренировочной части

(127656,)

In [26]:
lemma_test.shape #проверка размерности тестовой части

(31915,)

In [27]:
target_train.sum() #колличество "токсичных" комментариев в тренировочной выборке

12980

In [28]:
target_test.sum() #колличество "токсичных" комментариев в тестовой выборке

3245

### Векторизация текстовых признаков

Для целей обучения инициализируем векторизатор текстовых признаков полученных выборок на основе TF-IDF оценки. 

In [29]:
#инициализация TF-IDF векторизатора с учетом исключения стоп-слов англоязычной лексики
count_tf_idf = TfidfVectorizer(stop_words=list(stop_words))

### Вывод

В ходе этапа подготовки данных проведены:
- Загрузка и ознакомление с предоставленным датасетом: размерность - 159570 строк и два столбца, содержащие тексты комметариев на английском языке и  значение признака "токсичности". Пропуски и полные дубликаты отсутсвуют. Наблюдается дисбаланс классов (из всего объема комментраиев только 11% "токсичные");
- Токенезация комментариев с предшествующим выделением регулярных выражение (только латиница во всех регистрах). В качестве единицы токенизации выбрано слово;
- Лемматизация с учетом части речи;
- Разделение исходных данных на обучающую и тестовую выборки в соотношении 4/1.
- Инициализация векторизатора текстов на базе TF-IDF оценки с учетов исключения стоп-слов англоязычной лексики.

## Обучение и выбор наилучшей модели

В качестве рассматриваемых моделей классификации обозначим следудующие: логистическая регрессия, линейная классификация методом опорных векторов, решающее дерево, случайный лес, градиентный бустинг (на базе LGBM классификатора).

Учтем наличие дисбаланса определяемых классов, по мере необходимости будем использовать в рассматриваемых моделях параметр балансировки весов классов.

Оценку качества моделей будем определять на базе среднего значения f1-метрики посредством кроссвалидации.

Для целей корректной векторизации текстовых признаков в процессах определения лучших параметров моделей по сетке и последующей кроссвалидации создадим функцию конвеера с пошаговым выполнением сначала векторизации, а затем вызова выбранной модели.

In [30]:
def pipe(model): #создание функции конвеера на основе принятого векторизатора текста и используемой модели
    steps = [('vectorizer', count_tf_idf), ('classifier', model)]
    return Pipeline(steps)

### Модель логистической регресии

In [31]:
parametrs_lr = {'classifier__C': [1, 5, 10, 15, 20, 30]} 

grid_lr = GridSearchCV(pipe(LogisticRegression(class_weight='balanced')), parametrs_lr, cv=3, scoring='f1')
grid_lr.fit(lemma_train, target_train)

С_lr = grid_lr.best_params_['classifier__C']

#model_lr = LogisticRegression(class_weight='balanced')

In [32]:
grid_lr.best_params_

{'classifier__C': 10}

In [33]:
model_lr = LogisticRegression(class_weight='balanced', C=С_lr)

In [34]:
f1_lr = cross_val_score(pipe(model_lr), lemma_train, target_train, cv=3, scoring='f1').mean()
f1_lr

0.7575213792863242

### Модель линейной классификации методом опорных векторов

In [35]:
model_lsvc = LinearSVC(class_weight='balanced')

In [36]:
f1_lsvc = cross_val_score(pipe(model_lsvc), lemma_train, target_train, cv=3, scoring='f1').mean()
f1_lsvc

0.7520836122028184

### Модель решающего дерева

In [37]:
#подбор гиперпараметров модели "решающего дерева"
parametrs_dt = {'classifier__max_depth': range (30, 51, 10),
                'classifier__class_weight': ['balanced', None]} 

grid_dt = GridSearchCV(pipe(DecisionTreeClassifier(random_state=12345)), parametrs_dt, cv=3, scoring='f1')
grid_dt.fit(lemma_train, target_train)

max_depth_dt = grid_dt.best_params_['classifier__max_depth']
class_weight_dt = grid_dt.best_params_['classifier__class_weight']

In [38]:
grid_dt.best_params_

{'classifier__class_weight': None, 'classifier__max_depth': 50}

In [39]:
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=max_depth_dt, class_weight=class_weight_dt)

In [40]:
f1_dt = cross_val_score(pipe(model_dt), lemma_train, target_train, cv=3, scoring='f1').mean()
f1_dt

0.6944100819011715

### Модель случайного леса

In [41]:
#подбор гиперпараметров модели "случайного леса"
parametrs_rf = {'classifier__n_estimators' : range (25, 76, 25),
                'classifier__max_depth': range (25, 51, 25),
                'classifier__class_weight': ['balanced', None]} 

grid_rf = GridSearchCV(pipe(RandomForestClassifier(random_state=12345)), parametrs_rf, cv=3, scoring='f1')
grid_rf.fit(lemma_train, target_train)

n_estimators_rf = grid_rf.best_params_['classifier__n_estimators']
max_depth_rf = grid_rf.best_params_['classifier__max_depth']
class_weight_rf = grid_rf.best_params_['classifier__class_weight']

In [42]:
grid_rf.best_params_

{'classifier__class_weight': 'balanced',
 'classifier__max_depth': 50,
 'classifier__n_estimators': 75}

In [43]:
model_rf = RandomForestClassifier(random_state=12345, n_estimators=n_estimators_rf, max_depth=max_depth_rf, class_weight=class_weight_rf)

In [44]:
f1_rf = cross_val_score(pipe(model_rf), lemma_train, target_train, cv=3, scoring='f1').mean()
f1_rf

0.4680389203553427

### Модель градиентного бустинга на базе LightGBM

In [45]:
parametrs_gb = {'classifier__num_leaves' :[30, 50],
                'classifier__n_estimators': [100, 150]}

grid_gb = GridSearchCV(pipe(LGBMClassifier(random_state=12345, class_weight='balanced')), parametrs_gb, cv=3, scoring='f1')
grid_gb.fit(lemma_train, target_train)

num_leaves_gb = grid_gb.best_params_['classifier__num_leaves']
n_estimators_gb = grid_gb.best_params_['classifier__n_estimators']

[LightGBM] [Info] Number of positive: 8654, number of negative: 76450
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.208733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 412393
[LightGBM] [Info] Number of data points in the train set: 85104, number of used features: 8023
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 8653, number of negative: 76451
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.077165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 413256
[LightGBM] [Info] Number of data points in the train set: 85104, number of used features: 8027
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0

In [46]:
grid_gb.best_params_

{'classifier__n_estimators': 150, 'classifier__num_leaves': 50}

In [47]:
model_gb = LGBMClassifier(random_state=12345, n_estimators=num_leaves_gb, num_leaves=num_leaves_gb, class_weight = 'balanced')

In [48]:
f1_gb = cross_val_score(pipe(model_gb), lemma_train, target_train, cv=3, scoring='f1').mean()
f1_gb

[LightGBM] [Info] Number of positive: 8654, number of negative: 76450
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.180270 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 412393
[LightGBM] [Info] Number of data points in the train set: 85104, number of used features: 8023
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 8653, number of negative: 76451
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.256642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 413256
[LightGBM] [Info] Number of data points in the train set: 85104, number of used features: 8027
[LightGB

0.7141258878884136

In [49]:
results = {'Model': ['Logistic Regression', 'LinearSVC', 'Decision Tree', 'Random Forest', 'LightGBM'], 
           'f1_score': [f1_lr, f1_lsvc, f1_dt, f1_rf, f1_gb]} 

### Вывод

In [50]:
models_results = pd.DataFrame(results)
models_results

,Model,f1_score
0,Logistic Regression,0.757521
1,LinearSVC,0.752084
2,Decision Tree,0.694410
3,Random Forest,0.468039
4,LightGBM,0.714126


В ходе исследования были подготовлены и обучены следудующие модели классификации: логистическая регрессия, линейная классификация методом опорных векторов, решающее дерево, случайный лес, градиентный бустинг (на базе LGBM классификатора). В результате исследования установлено, что наилучшей средней метрикой f1, полученной на кроссвалидации является модель логистической регресии с парамтерами балансировки весов классов и параметром С=10. Наихудшие результаты у модели случайного леса.

## Тестирование наилучшей модели

### Тестирование модели

Проведем тестирование наилучшей модели (модели логистической регресии) на тестовой выборке данных.

Проведем векторизацию тектстовых признаков для обучающей и тестовой выборки.

In [51]:
tf_idf_train = count_tf_idf.fit_transform(lemma_train)
tf_idf_test = count_tf_idf.transform(lemma_test)

Проведем обучение модели на обучающей выборке.

In [52]:
best_model = model_lr
best_model.fit(tf_idf_train, target_train)

LogisticRegression(C=10, class_weight='balanced')

Определим прогнозные значения полученной модели на тестовой выборке

In [53]:
prediction = best_model.predict(tf_idf_test)

Определим качество полученного прогноза на основании f1 метрики.

In [54]:
f1_score(target_test, prediction)

0.7569335798606173

### Вывод

Тестирование наилучшей модели (модели логистической регресии) на тестовой выборке данных показало, что модель удовлетворяет заданным значениям f1 метрики: не менее 0,75. Таким образом, модель обладает достаточным качеством.

## Выводы

Цель исследования: создание и обучение модели классификации для разделения текстовых комментариев описания товаров интернет-магазина на позитивные и негативные.

Целевая метрика качества: *f1* не меньше 0.75. 

В ходе этапа подготовки данных проведены:
- Загрузка и ознакомление с предоставленным датасетом. Пропуски и полные дубликаты отсутсвуют. Наблюдается дисбаланс классов (из всего объема комментраиев только 11% "токсичные");
- Токенезация комментариев;
- Лемматизация с учетом части речи;
- Разделение исходных данных на обучающую и тестовую выборки в соотношении 4/1.
- Инициализация векторизатора текстов на базе TF-IDF оценки с учетов исключения стоп-слов англоязычной лексики.


В ходе исследования были подготовлены и обучены следудующие модели классификации: логистическая регрессия, линейная классификация методом опорных векторов, решающее дерево, случайный лес, градиентный бустинг (на базе LGBM классификатора). В результате исследования установлено, что наилучшей средней метрикой f1, полученной на кроссвалидации является модель логистической регресиии.

Тестирование наилучшей модели показало ее соответсвие целевой метрике качества: *f1* не меньше 0.75 (f1=0.762).